In [131]:
# standard libraries
from __future__ import print_function
import os
import re
import sys
from pathlib import Path
from enum import Enum
import json

# pysat library
from pysat.solvers import Solver, SolverNames
from pysat.formula import CNF, WCNF
from pysat.examples.fm import FM
from pysat.examples.musx import MUSX

# or-tools library
from ortools.linear_solver import pywraplp

# numpy
import numpy as np

# configs
import pprint
ppprint = pprint.PrettyPrinter(indent=4).pprint
def debug(info, verbose=False):
    if verbose:
        print(info)

def debug_ppprint(info, verbose=False):
    ppprint(json.dumps(info, indent=4))

In [104]:
folderPaths={
    'easyInstances':'/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/easy_instances/',
    'instance':'/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/instance/',
    'aaaiInstances':'/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/hard_instances/aaai_instances',
    'isingModel':'/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/hard_instances/Generalized_Ising_model',
    'maxSat':'/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/hard_instances/maxsat_staffscheduling_instances',
    'circuitDebugging':'/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/hard_instances/ms_industrial/circuit-debugging-problems',
    'safarpour':'/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/hard_instances/ms_industrial/sean-safarpour'
}

class Difficulty(Enum):
    EASY = 1
    MEDIUM = 2
    HARD = 3
    ALL = 0

def instanceDiff(fileSize):
    
    mb = 10* 1024 ** 1
    mediumUb = 10 * mb
    if fileSize < mb:
        return Difficulty.EASY
    elif fileSize < mediumUb:
        return Difficulty.MEDIUM
    else:
        return Difficulty.HARD

    
def allInstances(difficulty, cnfExtensions=['.cnf', '.wcnf']):
    instances = []
    for folder in folderPaths:
        instanceFolder = Path(folderPaths[folder])
        instances += [x for x in instanceFolder.iterdir() if x.is_file() and x.suffix in cnfExtensions]
    
    if difficulty is Difficulty.ALL:
        return instances

    sizeFilteredInstances = list(filter(lambda x: instanceDiff(x.stat().st_size) is difficulty, instances))

    return sizeFilteredInstances
    
def getDataPaths(cnfExtensions=['.cnf', '.wcnf'], difficulty= Difficulty.EASY):
    
    if difficulty not in Difficulty:
        ppprint('Difficulty must be in ' +str(difficulties) + ' defaulting to easy.')
        difficulty = Difficulty.EASY
    
    instances = allInstances(difficulty, cnfExtensions)

    return instances


In [105]:
def cnfInstances(difficulty=Difficulty.EASY):
    instances = [instance for instance in getDataPaths(cnfExtensions=['.cnf'], difficulty= difficulty)] 
    return instances

def wcnfInstances(difficulty=Difficulty.EASY):
    instances = [instance for instance in getDataPaths(cnfExtensions=['.wcnf'], difficulty= difficulty)] 
    return instances

In [106]:
def CNFisUnsat(instance, verbose=True):
    with Solver(name = SolverNames.minisat22[0]) as s:
        cnf = CNF(from_file=instance)
        added = s.append_formula(cnf.clauses, no_return=False)
        solved = s.solve()
    return solved

In [107]:
def WCNFisUnsat(instance, verbose=True):
    with Solver(name = SolverNames.minisat22[0]) as s:
        wcnf = WCNF(from_file=instance)
        added = s.append_formula(wcnf.clauses, no_return=False)
        solved = s.solve()
    return solved

In [110]:
def cnfUnsatInstances():
    return [instance  for instance in cnfInstances() if CNFisUnsat(instance)]

In [136]:
def get_constraint_coefficients(H, num_vars):
    hij = []
    for h in H:
        hj = []
        for e in range(1, num_vars+1):
            if e in h:
                hj.append(1)
            else:
                hj.append(0)
        hij.append(hj)           
    
    return hij


In [146]:
def create_data_model(H, weights, num_vars):
    """Stores the data for the problem."""
    data = {}
    data['constraint_coefficients'] = get_constraint_coefficients(H, num_vars)
    data['bounds'] = list(np.ones(len(H), dtype=np.int8))
    data['obj_coeffs'] = get_obj_coefficients(H, weights)
    data['num_vars'] = num_vars
    data['num_constraints'] = len(H)
    return data

In [151]:
def f(x):
    # weighted based on the number of literals inside the clause
    return len(x)

In [154]:
def cnf_weights(cnf):
    return [f(clause) for clause in cnf.clauses]

In [ ]:
def get_obj_coefficients(H,weights):
    indices = set()
    for hj in H:
        for i in hj:
            indices.add(i)   
    obj_coeffs = {}
    for index in indices:
        obj_coeffs[index] = weights[index]
    
    return obj_coeffs

In [157]:
smus_cnf = smus_CNF()
weights = cnf_weights(smus_cnf )
#print(weights)

h_test = [
    [3],
    [2, 4],
    [5, 4],
    [3, 5, 6]
]
print(create_data_model(h_test, 8))

False


AttributeError: 'list' object has no attribute 'clauses'

In [ ]:
def optimalHittingSet(H, num_vars, f, weights):
    indices = set()
    data = create_data_model(H, num_vars)
    # [START solver]
    # Create the mip solver with the CBC backend.
    solver = pywraplp.Solver('OptimalHittingSet', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    # [END solver]
    
    # [START variables]
    #infinity = solver.infinity()
    x = {}
    for j in range(1, data['num_vars']+1):
        x[j] = solver.IntVar(0, 1, 'x[%i]' % j)
    # [END variables]
    
    # [START constraints]
    for i in range(data['num_constraints']):
        #constraint = solver.RowConstraint(0, data['bounds'][i], '')
        #for j in range(1, data['num_vars']+1):
        #    constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j-1])
        #    print('Number of constraints =', solver.NumConstraints())
        constraint_expr = [data['constraint_coeffs'][i][j-1] * x[j] for j in range(1, data['num_vars']+1)]
        solver.Add(sum(constraint_expr) >= data['bounds'][i])


    # [END constraints]
    
    # [START objective]
    # Maximize 2*x + 2*y + 3*z
    objective = solver.Objective()
    for j in range(1, data['num_vars']+1):
        objective.SetCoefficient(x[j], data['obj_coeffs'][j-1])
    objective.SetMinimization()
    # [END objective]
    
    # Solve the problem and print the solution.
    # [START print_solution]
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        for j in range(1, data['num_vars']+1):
            print(x[j].name(), ' = ', x[j].solution_value())
        print()
        print('Problem solved in %f milliseconds' % solver.wall_time())
        print('Problem solved in %d iterations' % solver.iterations())
        print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    else:
        print('The problem does not have an optimal solution.')

    # [END print_solution]

    
    ohs = []
    
    return ohs

In [155]:
def flatten_set(H):
    return list(set([val for sublist in H for val in sublist]))

In [120]:
def omus(cnf):
    clauses = cnf.clauses
    H = []
    
    
    return 

In [128]:
def smus_CNF():
    l = 1
    m = 2
    n = 3
    p = 4
    s = 5
    cnf = CNF()
    cnf.append([-s])    # c1: ¬s
    cnf.append([s, -p]) # c2: s or ¬p
    cnf.append([p])     # c3: p
    cnf.append([-p, m]) # c4: ¬p or m
    cnf.append([-m, n]) # c5: ¬m or n
    cnf.append([-n])    # c6: ¬n
    cnf.append([-m, l]) # c7 ¬m or l
    cnf.append([-l])    # c8 ¬l
    
    return cnf

In [125]:
cnfs = sorted(cnfUnsatInstances(), key=lambda item: item.stat().st_size)[:3]
for c in cnfs:
    cnf = CNF(from_file=c)
    o = omus(cnf)
    print(c, c.stat().st_size)
    print(cnf.clauses)

# useless to call mus on cnf files, only on WCNF
#for cnf_name, cnf_dict in cnfs.items():
#    wcnf = CNF(from_file = cnf_dict['path']).weighted()
#    with MUSX(wcnf, verbosity=1) as musx:
#        print(musx.compute())
#wncf = WCNF(from_file = cnf1['path'])

/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/easy_instances/simple_v3_c2.cnf 48
[[1, -3], [2, 3, -1]]
/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/easy_instances/quinn.cnf 243
[[1, 2], [-2, -4], [3, 4], [-4, -5], [5, -6], [6, -7], [6, 7], [7, -16], [8, -9], [-8, -14], [9, 10], [9, -10], [-10, -11], [10, 12], [11, 12], [13, 14], [14, -15], [15, 16]]
/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/instance/reg_50_100.cnf 1077
[[1, -50, -32], [1, -50, -27], [1, -49, -22], [1, -49, 46], [1, -48, -15], [1, -48, 34], [1, -47, 7], [1, -47, 27], [1, -46, -36], [1, -46, 7], [1, -45, 44], [1, -45, 46], [1, -44, -5], [1, -44, 24], [1, -43, 18], [1, -43, 42], [1, -42, -41], [1, -42, 26], [1, -41, -21], [1, -40, 15], [1, -40, 20], [1, -39, 33], [1, -39, 43], [1, -38, 24], [1, -38, 36], [1, -37, -12], [1, -37, 25], [1, -36, 45], [1, -35, 21], [1, -35, 39], [1, -34, 8], [1, -34, 23], [1, -33, -12], [1, -33, 16], [1, -32, 4], [1, 

In [87]:
wcnf = WCNF(from_file='/home/crunchmonster/Documents/VUB/02_Research/02_Notes/02_OMUS/B_data/hard_instances/maxsat_staffscheduling_instances/'+'instance1.wcnf')
with MUSX(wcnf, verbosity=1) as musx:
    print(musx.compute())

c MUS approx: 15 45 53 71 102 114 119 120 122 123 0
[15, 45, 53, 71, 102, 114, 119, 120, 122, 123]


[1, 1, 1, 1, 1]